In [1]:
import numpy as np
import pickle as pkl
import torch
from scipy.spatial.distance import cosine, euclidean
import pandas as pd
import pprint as pp

In [2]:
import seaborn as sns
import matplotlib.pyplot as plt

In [3]:
from sklearn.preprocessing import StandardScaler

In [4]:
%matplotlib inline

In [5]:
import bz2
import _pickle as cPickle

# Load any compressed pickle file
def decompress_pickle(file):
    assert file.endswith("pbz2")
    data = bz2.BZ2File(file, "rb")
    data = cPickle.load(data,encoding='latin1')
    return data

In [6]:
#data

In [7]:
run_no = "1"

In [8]:
is_abs_A = False

In [9]:
cc_start_entity = "d" # d, r

In [10]:
if run_no == "1":
    data_version_name = "5"
    dname_data = "./../../data/ade"+run_no+"/"
    fname_data = dname_data+"dict_nsides_mimic_data_v"+data_version_name+"_case1_part.pkl.pbz2"
elif run_no == "2":
    data_version_name = "1"
    dname_data = "./../../data/ade"+run_no+"/"
    fname_data = dname_data+"dict_nsides_mimic_data_v"+data_version_name+"_case1_part.pkl.pbz2"    
elif run_no == "3":
    data_version_name = "2"
    dname_data = "./../../data/ade"+run_no+"/"
    fname_data = dname_data+"dict_nsides_mimic_data_v"+data_version_name+"_case1_part.pkl.pbz2"    
elif run_no == "4":
    data_version_name = "4"
    dname_data = "./../../data/ade"+run_no+"/"
    fname_data = dname_data+"dict_nsides_mimic_data_v"+data_version_name+"_case1_part.pkl.pbz2"

In [11]:
fname_data

'./../../data/ade1/dict_nsides_mimic_data_v5_case1_part.pkl.pbz2'

In [12]:
#data_dict = pkl.load(open(fname_data,"rb"))
data_dict = decompress_pickle(fname_data)

In [13]:
data_dict.keys()

dict_keys(['matrices_data', 'metadata', 'gt_case1_part'])

In [14]:
data_dict["metadata"].keys()

dict_keys(['dict_p_id_idx_map', 'dict_r_id_idx_map', 'dict_d_id_idx_map', 'dict_p_idx_id_map', 'dict_r_idx_id_map', 'dict_d_idx_id_map'])

In [15]:
dict_mat = data_dict["matrices_data"]
X_pd = dict_mat["mat_pat_dis_treat"]
X_pr = dict_mat["mat_pat_drugs"]
X_rd = dict_mat["mat_drugs_dis_side"]

In [16]:
G = {
    "p":["X_pr","X_pd"],\
    "r":["X_pr","X_rd"],\
    "d":["X_rd","X_pd"]}

X_data = {
    "X_pr":X_pr, \
    "X_pd":X_pd, \
    "X_rd":X_rd
    }

X_meta = {
    "X_pr":["p","r"],
    "X_pd":["p","d"],
    "X_rd":["r","d"]}

X_dtype = {
    "X_pr":"real", \
    "X_pd":"real", \
    "X_rd":"real"
}
    
#dict_num_clusters = {'p': 5, 'r': 10, 'd': 9} 
if data_version_name in ["1"]:
    dict_num_clusters =  {'p': 4, 'r': 10, 'd': 10}
elif data_version_name in ["2"]:
    dict_num_clusters =  {'p': 5, 'r': 10, 'd': 10}
elif data_version_name in ["3"]:
    dict_num_clusters =  {'p': 4, 'r': 8, 'd': 10}
elif data_version_name in ["4"]:
    dict_num_clusters =  {'p': 5, 'r': 10, 'd': 9} 
elif data_version_name in ["5"]:
    dict_num_clusters =  {'p': 5, 'r': 10, 'd': 9} 
else:
    assert False,"Unknown adr_data_version_name: "+data_version_name

In [17]:
dict_num_clusters

{'p': 5, 'r': 10, 'd': 9}

In [18]:
#out

In [19]:
dname_out = "./../../out_clust/ade"+run_no+"/version_"+data_version_name+"/"

In [20]:
dname_out

'./../../out_clust/ade1/version_5/'

In [21]:
fname_clust_labels = dname_out+"dict_c_clust_labels.pkl"
fname_A = dname_out+"dict_A.pkl"
fname_X_prime = dname_out+"dict_recons_X.pkl"
fname_Y_prime = dname_out+"dict_recons_Y.pkl"
fname_I_ortho = dname_out+"dict_I_ortho.pkl"
fname_U = dname_out+"dict_U.pkl"
fname_mu = dname_out+"dict_mu.pkl"

In [22]:
dict_clust_labels = pkl.load(open(fname_clust_labels,"rb"))
dict_A = pkl.load(open(fname_A,"rb"))
dict_X_prime = pkl.load(open(fname_X_prime,"rb"))
dict_Y_prime = pkl.load(open(fname_Y_prime,"rb"))
dict_I_ortho = pkl.load(open(fname_I_ortho,"rb"))
dict_U = pkl.load(open(fname_U,"rb"))
dict_mu = pkl.load(open(fname_mu,"rb"))

In [23]:
# dict_A = {}
# for x_id in dict_A_raw.keys():
#     scaler1 = StandardScaler()
#     dict_A[x_id] = scaler1.fit_transform(dict_A_raw[x_id])

In [24]:
#sanity check - start

In [25]:
dict_A.keys()

dict_keys(['X_pr', 'X_pd', 'X_rd'])

In [26]:
dict_A["X_rd"]

array([[-117.99486  , -181.4257   ,  -19.596973 ,  -51.72543  ,
         -26.278326 ,  -33.840935 ,  -23.20325  ,  -62.45585  ,
         -28.142876 ],
       [ 115.74491  ,   73.25941  ,   17.841402 ,   27.881687 ,
          19.318071 ,   23.917866 ,   20.161655 ,   35.32582  ,
          17.993277 ],
       [  28.70086  ,   33.531677 ,    4.7385087,   13.3003235,
           4.6696253,    6.891186 ,    4.7402506,   12.167971 ,
           9.751493 ],
       [ -21.943996 ,    0.6731851,   -2.493927 ,   -4.2688174,
          -2.494796 ,   -3.1439736,   -4.1078534,   -1.5874348,
          -1.7532332],
       [  53.07267  ,   26.22287  ,    8.024763 ,   13.859955 ,
           7.663022 ,   10.965383 ,    8.40477  ,   14.717553 ,
           8.031314 ],
       [   5.8237967,   55.08526  ,    2.1822042,    8.991854 ,
           5.7856693,    7.9529104,    5.267193 ,   18.606174 ,
           5.1338243],
       [ 211.50674  ,   63.269394 ,   27.338894 ,   36.250263 ,
          29.830524 ,   35.045

In [27]:
np.round(dict_A["X_rd"])

array([[-118., -181.,  -20.,  -52.,  -26.,  -34.,  -23.,  -62.,  -28.],
       [ 116.,   73.,   18.,   28.,   19.,   24.,   20.,   35.,   18.],
       [  29.,   34.,    5.,   13.,    5.,    7.,    5.,   12.,   10.],
       [ -22.,    1.,   -2.,   -4.,   -2.,   -3.,   -4.,   -2.,   -2.],
       [  53.,   26.,    8.,   14.,    8.,   11.,    8.,   15.,    8.],
       [   6.,   55.,    2.,    9.,    6.,    8.,    5.,   19.,    5.],
       [ 212.,   63.,   27.,   36.,   30.,   35.,   29.,   39.,   22.],
       [  -1.,   14.,    3.,    5.,    2.,    2.,    2.,    4.,    1.],
       [ -69.,  -96.,  -11.,  -29.,  -15.,  -19.,  -14.,  -34.,  -16.],
       [-149.,  -31.,  -21.,  -22.,  -21.,  -26.,  -23.,  -29.,  -17.]],
      dtype=float32)

In [28]:
A_sr = np.abs(dict_A["X_rd"].T)
scaler1 = StandardScaler()
A_sr = scaler1.fit_transform(A_sr)
#
A_temp = np.zeros(A_sr.shape)
for i in np.arange(A_temp.shape[0]):
    j = np.argmax(A_sr[i,:])
    A_temp[i,j] = 1
A_temp    

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0., 0.]])

In [29]:
np.round(A_sr,2).T

array([[ 1.11,  2.34, -0.79, -0.17, -0.66, -0.52, -0.72,  0.04, -0.63],
       [ 2.42,  1.08, -0.67, -0.35, -0.62, -0.48, -0.6 , -0.12, -0.66],
       [ 1.53,  2.01, -0.83,  0.01, -0.84, -0.62, -0.83, -0.1 , -0.34],
       [ 2.78, -0.65, -0.36, -0.07, -0.36, -0.25, -0.1 , -0.51, -0.48],
       [ 2.6 ,  0.68, -0.63, -0.21, -0.65, -0.42, -0.6 , -0.15, -0.63],
       [-0.44,  2.72, -0.68, -0.24, -0.45, -0.31, -0.48,  0.38, -0.49],
       [ 2.77,  0.15, -0.49, -0.33, -0.44, -0.35, -0.45, -0.29, -0.58],
       [-0.76,  2.71, -0.26,  0.2 , -0.48, -0.4 , -0.39,  0.05, -0.66],
       [ 1.28,  2.24, -0.8 , -0.18, -0.67, -0.53, -0.71,  0.02, -0.65],
       [ 2.81, -0.18, -0.41, -0.4 , -0.42, -0.29, -0.37, -0.21, -0.53]],
      dtype=float32)

In [30]:
A_sr = np.abs(dict_A["X_pr"])
A_temp = np.zeros(A_sr.shape)
for i in np.arange(A_temp.shape[0]):
    j = np.argmax(A_sr[i,:])
    A_temp[i,j] = 1
A_temp    

array([[0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0.]])

In [31]:
A_sr = np.abs(dict_A["X_pd"])
# scaler1 = StandardScaler()
# A_sr = scaler1.fit_transform(A_sr)
A_temp = np.zeros(A_sr.shape)
for i in np.arange(A_temp.shape[0]):
    j = np.argmax(A_sr[i,:])
    A_temp[i,j] = 1
A_temp    

array([[0., 0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0.]])

In [32]:
np.round(np.dot(dict_I_ortho["r"].T, dict_I_ortho["r"]))

array([[ 1., -0.,  0.,  0.,  0., -0.,  0., -0.,  0.,  0.],
       [-0.,  1., -0., -0.,  0.,  0., -0., -0.,  0., -0.],
       [ 0., -0.,  1., -0., -0.,  0., -0.,  0.,  0., -0.],
       [ 0., -0., -0.,  1., -0.,  0., -0., -0.,  0.,  0.],
       [ 0.,  0., -0., -0.,  1., -0., -0.,  0.,  0.,  0.],
       [-0.,  0.,  0.,  0., -0.,  1., -0.,  0., -0., -0.],
       [ 0., -0., -0., -0., -0., -0.,  1.,  0., -0., -0.],
       [-0., -0.,  0., -0.,  0.,  0.,  0.,  1., -0.,  0.],
       [ 0.,  0.,  0.,  0.,  0., -0., -0., -0.,  1., -0.],
       [ 0., -0., -0.,  0.,  0., -0., -0.,  0., -0.,  1.]], dtype=float32)

In [33]:
np.round(np.dot(dict_I_ortho["d"].T, dict_I_ortho["d"]))

array([[ 1.,  0., -0., -0., -0., -0., -0.,  0., -0.],
       [ 0.,  1., -0., -0.,  0.,  0.,  0.,  0., -0.],
       [-0., -0.,  1., -0.,  0.,  0.,  0., -0.,  0.],
       [-0., -0., -0.,  1., -0.,  0., -0.,  0., -0.],
       [-0.,  0.,  0., -0.,  1.,  0.,  0.,  0.,  0.],
       [-0.,  0.,  0.,  0.,  0.,  1., -0., -0.,  0.],
       [-0.,  0.,  0., -0.,  0., -0.,  1., -0.,  0.],
       [ 0.,  0., -0.,  0.,  0., -0., -0.,  1., -0.],
       [-0., -0.,  0., -0.,  0.,  0.,  0., -0.,  1.]], dtype=float32)

In [34]:
#sanity check - end

In [35]:
dict_clust_labels

{'p': array([0, 0, 0, ..., 0, 0, 0], dtype=int32),
 'r': array([2, 6, 8, 1, 0, 9, 5, 3, 9, 9, 2, 4, 4, 5, 9, 1, 1, 5, 9, 1, 0, 3,
        9, 9, 3, 5, 1, 5, 9, 2, 0, 9, 1, 9, 9, 9, 5, 9, 9, 1, 5, 4, 5, 9,
        8, 2, 1, 0, 1, 2, 1, 3, 2, 4, 1, 1, 7, 2, 2, 9, 8, 5, 2, 5, 4, 9,
        9, 9, 8, 5, 4, 6, 4, 0, 1, 4, 1, 9, 7, 4, 5, 9, 1, 9, 2, 9, 9, 4,
        1, 5, 4, 1, 1, 1, 1, 9, 0, 5, 6, 9, 1, 9, 0, 9, 9, 9, 9, 6, 9, 5,
        1, 6, 6, 4, 4, 1, 5, 1, 2, 9, 0, 9, 9, 2, 4, 1, 5, 4, 5, 0, 9, 1,
        5, 8, 1, 1, 1, 5, 1, 4, 9, 1, 0, 2, 9, 5, 5, 9, 2, 8, 9, 5, 4, 4,
        1, 9, 9, 9, 9, 6, 9, 5, 9, 3, 4, 7, 7, 9, 4, 0, 7, 9, 4, 1, 3, 9,
        9, 0, 9, 0, 1, 9, 1, 4, 4, 1, 2, 1, 6, 5, 9, 8, 1, 5, 5, 8, 1, 5,
        0, 5, 9, 2, 0, 5, 9, 1, 9, 2, 9, 5, 5, 1, 8, 2, 1, 1, 1, 1, 9, 1,
        1, 5, 2, 9, 9, 1, 2, 1, 5, 0, 9, 5, 6, 5, 5, 5, 9, 5, 1, 0, 6, 5,
        8, 2, 5, 2, 8, 4, 5, 5, 9, 9, 2, 5, 5, 9, 5, 2, 1, 0, 9, 6, 6, 8,
        9, 5, 0, 6, 2, 1, 1, 6, 8, 9, 4, 2, 0, 4, 8, 1, 

In [36]:
def get_cluster_num_idx(dict_clust_labels,e_id,num_clusters):
    p_clust_id_list = dict_clust_labels[e_id]
#     p_clust_id_list = []
#     for i in np.arange(U.shape[0]):
#         temp_arr = U[i]
#         p_clust_id_list.append(np.argmax(temp_arr))
    #
    p_clust_id_num_dict = {}
    p_clust_id_idx_dict = {}
    for j in np.arange(num_clusters[e_id]):
        temp_clust_j = np.array(p_clust_id_list) == j
        temp_clust_j_idx = np.arange(p_clust_id_list.shape[0])[temp_clust_j]
        num_p = np.sum(temp_clust_j)
        print("cluster: ",j,", #", num_p)
        p_clust_id_num_dict[j] = num_p
        p_clust_id_idx_dict[j] = temp_clust_j_idx     
    return p_clust_id_num_dict, p_clust_id_idx_dict,p_clust_id_list

In [37]:
dict_entity_cluster_info = {}
for e_id in G.keys():
    print("e_id: ",e_id)
    temp_num, temp_idx, temp_pred_idx = get_cluster_num_idx(dict_clust_labels,e_id,dict_num_clusters)
    dict_entity_cluster_info[e_id] = {"num":temp_num,"idx":temp_idx,"temp_pred_idx":temp_pred_idx}

e_id:  p
cluster:  0 , # 4948
cluster:  1 , # 548
cluster:  2 , # 15
cluster:  3 , # 329
cluster:  4 , # 50
e_id:  r
cluster:  0 , # 54
cluster:  1 , # 110
cluster:  2 , # 48
cluster:  3 , # 12
cluster:  4 , # 44
cluster:  5 , # 101
cluster:  6 , # 38
cluster:  7 , # 12
cluster:  8 , # 22
cluster:  9 , # 155
e_id:  d
cluster:  0 , # 786
cluster:  1 , # 330
cluster:  2 , # 13
cluster:  3 , # 49
cluster:  4 , # 20
cluster:  5 , # 28
cluster:  6 , # 19
cluster:  7 , # 57
cluster:  8 , # 19


In [38]:
dict_A

{'X_pr': array([[-7.1947575e+00, -8.9039373e+00, -1.8243607e+01, -3.2192874e+00,
         -1.6628550e+01, -5.5368519e+00, -7.0359082e+00, -2.7110577e+00,
         -7.0509338e+01, -9.2836838e+00],
        [ 6.4393501e+00, -5.2363143e+00,  2.0530801e+00, -1.5820880e+00,
         -3.5911882e+00, -1.6372939e+00, -6.3619285e+00, -8.9343739e-01,
          2.0981097e+02, -4.7007942e-01],
        [ 4.6913058e-02,  2.9809308e+00,  3.9559836e+00,  7.1643293e-01,
          2.3431404e+00,  1.8765232e-01,  5.3407639e-02,  1.1061440e+00,
         -2.9608154e+00,  3.3300931e+00],
        [ 1.8442781e+01,  3.8308506e+01,  6.4280251e+01,  1.4619688e+01,
          6.2241661e+01,  2.0689098e+01,  3.2704689e+01,  1.0937341e+01,
          3.8918591e+00,  3.3678062e+01],
        [ 2.9201927e+00,  6.0106025e+00,  7.6327152e+00, -6.3144910e-01,
          1.6364826e+01,  7.3268833e+00,  7.1321907e+00,  1.2652466e+00,
         -1.5422033e+00,  5.6957903e+00]], dtype=float32),
 'X_pd': array([[-4.65656853e+00, -

In [39]:
# dict_p_cluster_asso_chain

In [40]:
#try 2

In [41]:
num_clusters = dict_num_clusters

#for every "p" cluster
dict_d_clust_num = dict_entity_cluster_info["d"]["num"]
dict_r_clust_num = dict_entity_cluster_info["r"]["num"]
dict_p_clust_num = dict_entity_cluster_info["p"]["num"]
#
dict_cc1 = {}
for d_clust_id in np.arange(num_clusters["d"]):
    if dict_d_clust_num[d_clust_id] > 0:
        #dr
        if is_abs_A:
            A_dr_temp = dict_A["X_rd"].T
            scaler1 = StandardScaler()
            A_dr_temp = np.abs(scaler1.fit_transform(A_dr_temp))
        else:
            A_dr_temp = dict_A["X_rd"].T
            scaler1 = StandardScaler()
            A_dr_temp = scaler1.fit_transform(A_dr_temp)
        r_clust_id_list = list(np.argsort(A_dr_temp[d_clust_id,:]))
        r_clust_id_list.reverse()
        for temp_r_id in r_clust_id_list:
            if dict_r_clust_num[temp_r_id] > 0:
                r_clust_id = temp_r_id
                break
    dict_cc1[d_clust_id] = {"d":d_clust_id, "r":r_clust_id}

dict_cc2 = {}
for d_clust_id in np.arange(num_clusters["d"]):
    if dict_d_clust_num[d_clust_id] > 0:
        #dp
        if is_abs_A:
            #A_dp_temp = np.abs(dict_A["X_pd"].T)
            A_dp_temp = dict_A["X_pd"].T
            scaler1 = StandardScaler()
            A_dp_temp = np.abs(scaler1.fit_transform(A_dp_temp))
        else:
            A_dp_temp = dict_A["X_pd"].T
            scaler1 = StandardScaler()
            A_dp_temp = scaler1.fit_transform(A_dp_temp)
        p_clust_id_list = list(np.argsort(A_dp_temp[d_clust_id,:]))
        p_clust_id_list.reverse()
        for temp_p_id in p_clust_id_list:
            if dict_p_clust_num[temp_p_id] > 0:
                p_clust_id = temp_p_id
                break        
        #pr
        if is_abs_A:
            A_pr_temp = dict_A["X_pr"]
            scaler1 = StandardScaler()
            A_pr_temp = np.abs(scaler1.fit_transform(A_pr_temp))
        else:
            A_pr_temp = dict_A["X_pr"]
            scaler1 = StandardScaler()
            A_pr_temp = scaler1.fit_transform(A_pr_temp)
        #A_pr_temp = dict_A["X_pr"]
        r_clust_id_list = list(np.argsort(A_pr_temp[p_clust_id,:]))
        r_clust_id_list.reverse()
        for temp_r_id in r_clust_id_list:
            if dict_r_clust_num[temp_r_id] > 0:
                r_clust_id = temp_r_id
                break
    dict_cc2[d_clust_id] = {"d":d_clust_id,"p":p_clust_id, "r":r_clust_id}        

In [42]:
dict_cc1

{0: {'d': 0, 'r': 6},
 1: {'d': 1, 'r': 5},
 2: {'d': 2, 'r': 8},
 3: {'d': 3, 'r': 9},
 4: {'d': 4, 'r': 8},
 5: {'d': 5, 'r': 8},
 6: {'d': 6, 'r': 0},
 7: {'d': 7, 'r': 3},
 8: {'d': 8, 'r': 8}}

In [43]:
dict_cc2

{0: {'d': 0, 'p': 1, 'r': 8},
 1: {'d': 1, 'p': 0, 'r': 3},
 2: {'d': 2, 'p': 2, 'r': 7},
 3: {'d': 3, 'p': 3, 'r': 3},
 4: {'d': 4, 'p': 4, 'r': 5},
 5: {'d': 5, 'p': 0, 'r': 3},
 6: {'d': 6, 'p': 0, 'r': 3},
 7: {'d': 7, 'p': 0, 'r': 3},
 8: {'d': 8, 'p': 1, 'r': 8}}

In [44]:
# for temp in dict_cc1.keys():
#     dict_cc1[temp].update(dict_cc2[temp])

In [45]:
# dict_temp_cc1 = dict_cc1

In [46]:
dict_temp_cc_d = {}
for temp_key in dict_cc1.keys():
    temp = {}
    cur_d = dict_cc1[temp_key]["d"]
    cur_r = dict_cc1[temp_key]["r"]
    temp["d"] = cur_d
    temp["r"] = cur_r
    for temp_key_2 in dict_cc2.keys():
        if cur_d == dict_cc2[temp_key_2]["d"]:
            cur_p = dict_cc2[temp_key_2]["p"]
            print("cur_d: ",cur_d,", cur_r: ",cur_r,", cur_p: ",cur_p)
            temp["p"] = cur_p
            dict_temp_cc_d[temp_key] = temp

cur_d:  0 , cur_r:  6 , cur_p:  1
cur_d:  1 , cur_r:  5 , cur_p:  0
cur_d:  2 , cur_r:  8 , cur_p:  2
cur_d:  3 , cur_r:  9 , cur_p:  3
cur_d:  4 , cur_r:  8 , cur_p:  4
cur_d:  5 , cur_r:  8 , cur_p:  0
cur_d:  6 , cur_r:  0 , cur_p:  0
cur_d:  7 , cur_r:  3 , cur_p:  0
cur_d:  8 , cur_r:  8 , cur_p:  1


In [47]:
dict_temp_cc_d

{0: {'d': 0, 'r': 6, 'p': 1},
 1: {'d': 1, 'r': 5, 'p': 0},
 2: {'d': 2, 'r': 8, 'p': 2},
 3: {'d': 3, 'r': 9, 'p': 3},
 4: {'d': 4, 'r': 8, 'p': 4},
 5: {'d': 5, 'r': 8, 'p': 0},
 6: {'d': 6, 'r': 0, 'p': 0},
 7: {'d': 7, 'r': 3, 'p': 0},
 8: {'d': 8, 'r': 8, 'p': 1}}

In [48]:
num_clusters = dict_num_clusters

#for every "p" cluster
dict_d_clust_num = dict_entity_cluster_info["d"]["num"]
dict_r_clust_num = dict_entity_cluster_info["r"]["num"]
dict_p_clust_num = dict_entity_cluster_info["p"]["num"]
#
dict_cc3 = {}
for r_clust_id in np.arange(num_clusters["r"]):
    if dict_r_clust_num[r_clust_id] > 0:
        #rs
        if is_abs_A:
            A_rd_temp = dict_A["X_rd"]
            scaler1 = StandardScaler()
            A_rd_temp = np.abs(scaler1.fit_transform(A_rd_temp))
        else:
            A_rd_temp = dict_A["X_rd"]
            scaler1 = StandardScaler()
            A_rd_temp = scaler1.fit_transform(A_rd_temp)
        d_clust_id_list = list(np.argsort(A_rd_temp[r_clust_id,:]))
        d_clust_id_list.reverse()
        for temp_d_id in d_clust_id_list:
            if dict_d_clust_num[temp_d_id] > 0:
                d_clust_id = temp_d_id
                break
    dict_cc3[r_clust_id] = {"r":r_clust_id, "d":d_clust_id}

dict_cc4 = {}
for r_clust_id in np.arange(num_clusters["r"]):
    if dict_r_clust_num[r_clust_id] > 0:
        #rp
        if is_abs_A:
            A_rp_temp = dict_A["X_pr"].T
            scaler1 = StandardScaler()
            A_rp_temp = np.abs(scaler1.fit_transform(A_rp_temp))
        else:
            A_rp_temp = dict_A["X_pr"].T
            scaler1 = StandardScaler()
            A_rp_temp = scaler1.fit_transform(A_rp_temp)
        #A_rp_temp = dict_A["X_pr"].T
        p_clust_id_list = list(np.argsort(A_rp_temp[r_clust_id,:]))
        p_clust_id_list.reverse()
        for temp_p_id in p_clust_id_list:
            if dict_p_clust_num[temp_p_id] > 0:
                p_clust_id = temp_p_id
                break
        #pd
        if is_abs_A:
            A_pd_temp = dict_A["X_pd"]
            scaler1 = StandardScaler()
            A_pd_temp = np.abs(scaler1.fit_transform(A_pd_temp))
        else:
            A_pd_temp = dict_A["X_pd"]
            scaler1 = StandardScaler()
            A_pd_temp = scaler1.fit_transform(A_pd_temp)
        d_clust_id_list = list(np.argsort(A_pd_temp[p_clust_id,:]))
        d_clust_id_list.reverse()
        for temp_d_id in d_clust_id_list:
            if dict_d_clust_num[temp_d_id] > 0:
                d_clust_id = temp_d_id
                break                 
    dict_cc4[r_clust_id] = {"d":d_clust_id, "p":p_clust_id, "r":r_clust_id}        

In [49]:
dict_cc3

{0: {'r': 0, 'd': 0},
 1: {'r': 1, 'd': 6},
 2: {'r': 2, 'd': 8},
 3: {'r': 3, 'd': 1},
 4: {'r': 4, 'd': 3},
 5: {'r': 5, 'd': 1},
 6: {'r': 6, 'd': 0},
 7: {'r': 7, 'd': 1},
 8: {'r': 8, 'd': 0},
 9: {'r': 9, 'd': 1}}

In [50]:
dict_cc4

{0: {'d': 2, 'p': 0, 'r': 0},
 1: {'d': 2, 'p': 2, 'r': 1},
 2: {'d': 3, 'p': 3, 'r': 2},
 3: {'d': 2, 'p': 0, 'r': 3},
 4: {'d': 4, 'p': 4, 'r': 4},
 5: {'d': 2, 'p': 0, 'r': 5},
 6: {'d': 2, 'p': 0, 'r': 6},
 7: {'d': 2, 'p': 0, 'r': 7},
 8: {'d': 0, 'p': 1, 'r': 8},
 9: {'d': 2, 'p': 2, 'r': 9}}

In [51]:
# for temp in dict_cc3.keys():
#     dict_cc3[temp].update(dict_cc4[temp])

In [52]:
dict_temp_cc_r = {}
for temp_key in dict_cc3.keys():
    temp = {}
    cur_d = dict_cc3[temp_key]["d"]
    cur_r = dict_cc3[temp_key]["r"]
    temp["d"] = cur_d
    temp["r"] = cur_r
    for temp_key_2 in dict_cc4.keys():
        if cur_r == dict_cc4[temp_key_2]["r"]:
            cur_p = dict_cc4[temp_key_2]["p"]
            temp["p"] = cur_p
            dict_temp_cc_r[temp_key] = temp

In [53]:
dict_temp_cc_d

{0: {'d': 0, 'r': 6, 'p': 1},
 1: {'d': 1, 'r': 5, 'p': 0},
 2: {'d': 2, 'r': 8, 'p': 2},
 3: {'d': 3, 'r': 9, 'p': 3},
 4: {'d': 4, 'r': 8, 'p': 4},
 5: {'d': 5, 'r': 8, 'p': 0},
 6: {'d': 6, 'r': 0, 'p': 0},
 7: {'d': 7, 'r': 3, 'p': 0},
 8: {'d': 8, 'r': 8, 'p': 1}}

In [54]:
dict_temp_cc_r

{0: {'d': 0, 'r': 0, 'p': 0},
 1: {'d': 6, 'r': 1, 'p': 2},
 2: {'d': 8, 'r': 2, 'p': 3},
 3: {'d': 1, 'r': 3, 'p': 0},
 4: {'d': 3, 'r': 4, 'p': 4},
 5: {'d': 1, 'r': 5, 'p': 0},
 6: {'d': 0, 'r': 6, 'p': 0},
 7: {'d': 1, 'r': 7, 'p': 0},
 8: {'d': 0, 'r': 8, 'p': 1},
 9: {'d': 1, 'r': 9, 'p': 2}}

In [55]:
# dict_temp_cc = {}
# count = 0
# for temp_key in dict_temp_cc1.keys():
#     dict_temp_cc[count] = dict_temp_cc1[temp_key]
#     count+=1
# for temp_key in dict_temp_cc2.keys():
#     dict_temp_cc[count] = dict_temp_cc2[temp_key]
#     count+=1    

In [56]:
if cc_start_entity == "r":
    dict_temp_cc = dict_temp_cc_r
elif cc_start_entity == "d":    
    dict_temp_cc = dict_temp_cc_d
else:
    assert False

In [57]:
dict_cluster_asso_chain = dict_temp_cc

In [58]:
dict_cluster_asso_chain

{0: {'d': 0, 'r': 6, 'p': 1},
 1: {'d': 1, 'r': 5, 'p': 0},
 2: {'d': 2, 'r': 8, 'p': 2},
 3: {'d': 3, 'r': 9, 'p': 3},
 4: {'d': 4, 'r': 8, 'p': 4},
 5: {'d': 5, 'r': 8, 'p': 0},
 6: {'d': 6, 'r': 0, 'p': 0},
 7: {'d': 7, 'r': 3, 'p': 0},
 8: {'d': 8, 'r': 8, 'p': 1}}

In [59]:
#try 2

In [60]:
from sklearn.metrics.pairwise import cosine_distances, cosine_similarity

In [61]:
def get_dist(X1, X2):
    d_list = np.diag(cosine_distances(X1,X2))
    #d = np.sqrt(np.sum(np.round(d_list,4)))
    d = np.mean(np.round(d_list,4))
    return d 

In [62]:
def get_dist_subspace(X1, X2, k, is_X1_orth=False, is_X2_orth=False):
    k1 = X1.shape[1]
    k2 = X2.shape[1]
    k = min(k1,k2)
    print("k1: ",k1,", k2: ",k2,", min(k1,k2): ",k)
    #
    if not is_X1_orth:
        u, s, vh = np.linalg.svd(X1)
        X1_tilde = u[:,0:k]
    if not is_X2_orth:
        u, s, vh = np.linalg.svd(X2)
        X2_tilde = u[:,0:k]
    #
    M = np.dot(X1_tilde.T, X2_tilde)
    #print("M.shape: ",M.shape)
    u, s, vh = np.linalg.svd(M)
    d = np.sqrt(np.sum(1 - np.round(pow(s,2),4)))
    #d = np.mean(1 - np.round(pow(s,2),2))
    return d    

In [63]:
dict_mu.keys()

dict_keys(['p_X_pr', 'p_X_pd', 'r_X_pr', 'r_X_rd', 'd_X_rd', 'd_X_pd'])

In [64]:
mu_dp = dict_mu["d_X_pd"]
mu_rp = dict_mu["r_X_pr"]

In [65]:
#X_rd = X_rd
X_dr = X_rd.T
#X_pr = X_pr
X_rp = X_pr.T
#X_pd = X_pd
X_dp = X_pd.T

In [66]:
dict_Y_prime.keys()

dict_keys(['p_X_pr', 'p_X_pd', 'r_X_pr', 'r_X_rd', 'd_X_rd', 'd_X_pd'])

In [67]:
X_rd2 = dict_Y_prime["r_X_rd"]
X_dr2 = dict_Y_prime["d_X_rd"]
X_pr2 = dict_Y_prime["p_X_pr"]
X_rp2 = dict_Y_prime["r_X_pr"]
X_pd2 = dict_Y_prime["p_X_pd"]
X_dp2 = dict_Y_prime["d_X_pd"]

In [68]:
X_rd_prime = dict_X_prime["X_rd"]
X_dr_prime = dict_X_prime["X_rd"].T
X_pr_prime = dict_X_prime["X_pr"]
X_rp_prime = dict_X_prime["X_pr"].T
X_pd_prime = dict_X_prime["X_pd"]
X_dp_prime = dict_X_prime["X_pd"].T

In [69]:
dict_entity_cluster_info.keys()

dict_keys(['p', 'r', 'd'])

In [70]:
dict_cluster_asso_chain[0].keys()

dict_keys(['d', 'r', 'p'])

In [71]:
dict_clust_id_score = {}
for temp_key in dict_cluster_asso_chain.keys():
    dict_cc = dict_cluster_asso_chain[temp_key]
    # 
    p_clust_id = dict_cc["p"]
    r_clust_id = dict_cc["r"]
    d_clust_id = dict_cc["d"]
    #
    p_idx_list = dict_entity_cluster_info["p"]["idx"][p_clust_id]
    r_idx_list = dict_entity_cluster_info["r"]["idx"][r_clust_id]
    d_idx_list = dict_entity_cluster_info["d"]["idx"][d_clust_id]
    #
#     X_rd_cc = X_rd[r_idx_list,:][:,d_idx_list]
#     X_dr_cc = X_dr[d_idx_list,:][:,r_idx_list]
#     X_pr_cc = X_pr[p_idx_list,:][:,r_idx_list]
#     X_rp_cc = X_rp[r_idx_list,:][:,p_idx_list]
#     X_pd_cc = X_pd[p_idx_list,:][:,d_idx_list]
#     X_dp_cc = X_dp[d_idx_list,:][:,p_idx_list]
    #
    X_rd_cc = X_rd2[r_idx_list,:][:,d_idx_list]
    X_dr_cc = X_dr2[d_idx_list,:][:,r_idx_list]
    X_pr_cc = X_pr2[p_idx_list,:][:,r_idx_list]
    X_rp_cc = X_rp2[r_idx_list,:][:,p_idx_list]
    X_pd_cc = X_pd2[p_idx_list,:][:,d_idx_list]
    X_dp_cc = X_dp2[d_idx_list,:][:,p_idx_list]
    #
    X_rd_prime_cc = X_rd_prime[r_idx_list,:][:,d_idx_list]
    X_dr_prime_cc = X_dr_prime[d_idx_list,:][:,r_idx_list]
    X_pr_prime_cc = X_pr_prime[p_idx_list,:][:,r_idx_list]
    X_rp_prime_cc = X_rp_prime[r_idx_list,:][:,p_idx_list]
    X_pd_prime_cc = X_pd_prime[p_idx_list,:][:,d_idx_list]
    X_dp_prime_cc = X_dp_prime[d_idx_list,:][:,p_idx_list]
    #
    mu_dp_cc = mu_dp[d_idx_list,:]
    mu_rp_cc = mu_rp[r_idx_list,:]
    #
    k = 20
    if cc_start_entity == "r":
        s1 = get_dist(X_rd_cc, X_rd_prime_cc)
        s21 = get_dist(X_rp_cc, X_rp_prime_cc)
        s22 = get_dist(X_pd_cc, X_pd_prime_cc)
        s2 = s21 + s22 
        s3 = get_dist_subspace(X_rp_cc, X_rd_prime_cc, k)
        #s3 = get_dist_subspace(mu_rp_cc, X_rd_prime_cc, k)
    elif cc_start_entity == "d":
        s1 = get_dist(X_dr_cc, X_dr_prime_cc)
        s21 = get_dist(X_dp_cc, X_dp_prime_cc)
        s22 = get_dist(X_pr_cc, X_pr_prime_cc)
        s2 = s21 + s22
        s3 = get_dist_subspace(X_dp_cc, X_dr_prime_cc, k)
        #s3 = get_dist_subspace(mu_dp_cc, X_dr_prime_cc, k)  
    #
    alpha = 1.0
    beta = 1.0
    gamma = 1.0
    cur_score = (alpha * s1) - (beta * s2) - (gamma * s3)
    #cur_score = (alpha * s1) + (beta * (1-s2)) + (gamma * (1-s3))
    dict_clust_id_score[temp_key] = cur_score
    print("cur_key: ",temp_key,"| s1: ",np.round(s1,4),", s2: ",np.round(s2,4),", s21: ",np.round(s21,4),", s22: ",np.round(s22,4),", s3: ",np.round(s3,4),", score: ",np.round(cur_score,4))

k1:  548 , k2:  38 , min(k1,k2):  38
cur_key:  0 | s1:  0.8922 , s2:  1.7518 , s21:  1.0135 , s22:  0.7384 , s3:  5.6712 , score:  -6.5308
k1:  4948 , k2:  101 , min(k1,k2):  101
cur_key:  1 | s1:  1.3787 , s2:  2.2108 , s21:  0.8861 , s22:  1.3247 , s3:  7.8243 , score:  -8.6565
k1:  15 , k2:  22 , min(k1,k2):  15
cur_key:  2 | s1:  1.2087 , s2:  0.6788 , s21:  0.2493 , s22:  0.4295 , s3:  0.0 , score:  0.5298
k1:  329 , k2:  155 , min(k1,k2):  155
cur_key:  3 | s1:  0.9398 , s2:  1.8858 , s21:  0.9548 , s22:  0.931 , s3:  0.0 , score:  -0.9461
k1:  50 , k2:  22 , min(k1,k2):  22
cur_key:  4 | s1:  0.6195 , s2:  2.6501 , s21:  1.0998 , s22:  1.5503 , s3:  0.0 , score:  -2.0306
k1:  4948 , k2:  22 , min(k1,k2):  22
cur_key:  5 | s1:  0.4413 , s2:  2.4279 , s21:  0.7456 , s22:  1.6823 , s3:  2.1611 , score:  -4.1477
k1:  4948 , k2:  54 , min(k1,k2):  54
cur_key:  6 | s1:  0.4345 , s2:  1.8474 , s21:  0.9454 , s22:  0.902 , s3:  0.0 , score:  -1.4129
k1:  4948 , k2:  12 , min(k1,k2):  12

In [72]:
dict_clust_id_score

{0: -6.530826210975647,
 1: -8.656460881233215,
 2: 0.5298349261283875,
 3: -0.9460525512695312,
 4: -2.0306090712547302,
 5: -4.147712886333466,
 6: -1.412877470254898,
 7: -3.7596726417541504,
 8: -1.9690729975700378}

In [73]:
keys_list = list(dict_clust_id_score.keys())
#val_list = np.abs(list(dict_clust_id_score.values()))
val_list = list(dict_clust_id_score.values())
max_score_clust_id = keys_list[np.argmax(val_list)]
max_cc = max_score_clust_id
print("max_key: ",max_score_clust_id)
print("max score:",dict_clust_id_score[max_score_clust_id])

max_key:  2
max score: 0.5298349261283875


In [74]:
keys_list = list(dict_clust_id_score.keys())
val_list = list(np.array(list(dict_clust_id_score.values())) * -1)
#val_list = np.abs(list(dict_clust_id_score.values())) * -1
max_score_clust_id = keys_list[np.argmax(val_list)]
min_cc = max_score_clust_id
print("min_key: ",max_score_clust_id)
print("min score:",dict_clust_id_score[max_score_clust_id])

min_key:  1
min score: -8.656460881233215


In [75]:
data_dict.keys()

dict_keys(['matrices_data', 'metadata', 'gt_case1_part'])

In [76]:
def get_overlap_percent(dict_entity_cluster_info,e_id,e_clust_id,case_no=1):
    if e_id in ["s","t"]:
        p_cur_idx = list(dict_entity_cluster_info["d"]['idx'][e_clust_id])
    else:
        p_cur_idx = list(dict_entity_cluster_info[e_id]['idx'][e_clust_id])
    p_gt_of_idx = data_dict["gt_case"+str(case_no)+"_part"][e_id+"_unknown_idx_list"] #gt_case1_part
    #p_gt_of_idx = data_dict["e_unknown_cluster_idx_perm"][e_id]
    p_overlap_idx = list(set(p_cur_idx).intersection(set(p_gt_of_idx)))
    p_per = np.round((len(p_overlap_idx)/len(p_gt_of_idx))*100.0,2)    
    return p_per

# dname_data2 = "C:\\Users\\mragu\\Documents\\heuristic\\NSIDES\\data\\adr_mimic_nsides\\"
# fname_data2 = dname_data2+"dict_nsides_mimic_data_v1.pkl"
# data_dict2 = pkl.load(open(fname_data2,"rb"))

# def get_overlap_percent2(dict_entity_cluster_info,e_id,e_clust_id,data_dict,case_no=2):
#     if e_id in ["s","t"]:
#         p_cur_idx = list(dict_entity_cluster_info["d"]['idx'][e_clust_id])
#     else:
#         p_cur_idx = list(dict_entity_cluster_info[e_id]['idx'][e_clust_id])
#     p_gt_of_idx = data_dict["gt_case"+str(case_no)][e_id+"_unknown_idx_list"] #gt_case1_part
#     #p_gt_of_idx = data_dict["e_unknown_cluster_idx_perm"][e_id]
#     p_overlap_idx = list(set(p_cur_idx).intersection(set(p_gt_of_idx)))
#     p_per = np.round((len(p_overlap_idx)/len(p_gt_of_idx))*100.0,2)    
#     return p_per

In [77]:
dict_cluster_asso_chain.keys()

dict_keys([0, 1, 2, 3, 4, 5, 6, 7, 8])

In [78]:
dict_cluster_asso_chain[0].keys()

dict_keys(['d', 'r', 'p'])

In [79]:
get_overlap_percent(dict_entity_cluster_info,"s",d_clust_id)

0.0

In [80]:
dict_result1 = {}
for temp_key in dict_cluster_asso_chain.keys():
    temp_res_dict = {}
    #
    p_clust_id = dict_cluster_asso_chain[temp_key]["p"]
    r_clust_id = dict_cluster_asso_chain[temp_key]["r"]
    s_clust_id = dict_cluster_asso_chain[temp_key]["d"]
    t_clust_id = dict_cluster_asso_chain[temp_key]["d"]
    #
    p_clust_size = dict_entity_cluster_info["p"]['num'][p_clust_id]
    r_clust_size = dict_entity_cluster_info["r"]['num'][r_clust_id]
    s_clust_size = dict_entity_cluster_info["d"]['num'][s_clust_id]
    t_clust_size = dict_entity_cluster_info["d"]['num'][t_clust_id]
    #
    p_overlap_per = get_overlap_percent(dict_entity_cluster_info,"p",p_clust_id)
    r_overlap_per = get_overlap_percent(dict_entity_cluster_info,"r",r_clust_id)
    s_overlap_per = get_overlap_percent(dict_entity_cluster_info,"s",s_clust_id)
    t_overlap_per = get_overlap_percent(dict_entity_cluster_info,"t",t_clust_id)
    #
    temp_res_dict["p_clust_id"] = p_clust_id
    temp_res_dict["r_clust_id"] = r_clust_id
    temp_res_dict["s_clust_id"] = s_clust_id
    temp_res_dict["t_clust_id"] = t_clust_id
    #
    temp_res_dict["p_clust_size"] = p_clust_size
    temp_res_dict["r_clust_size"] = r_clust_size
    temp_res_dict["s_clust_size"] = s_clust_size
    temp_res_dict["t_clust_size"] = t_clust_size
    #
    temp_res_dict["p_overlap_per"] = p_overlap_per
    temp_res_dict["r_overlap_per"] = r_overlap_per
    temp_res_dict["s_overlap_per"] = s_overlap_per
    temp_res_dict["t_overlap_per"] = t_overlap_per
    #
    dict_result1[temp_key] = temp_res_dict

In [81]:
df_result1 = pd.DataFrame(dict_result1)
pp.pprint(df_result1)
print("#")
df_result1 = pd.DataFrame(dict_result1).T
#pp.pprint(df_result[["p_overlap_per","r_overlap_per","s_overlap_per","t_overlap_per"]].T)

                    0        1      2       3      4        5        6  \
p_clust_id       1.00     0.00   2.00    3.00   4.00     0.00     0.00   
r_clust_id       6.00     5.00   8.00    9.00   8.00     8.00     0.00   
s_clust_id       0.00     1.00   2.00    3.00   4.00     5.00     6.00   
t_clust_id       0.00     1.00   2.00    3.00   4.00     5.00     6.00   
p_clust_size   548.00  4948.00  15.00  329.00  50.00  4948.00  4948.00   
r_clust_size    38.00   101.00  22.00  155.00  22.00    22.00    54.00   
s_clust_size   786.00   330.00  13.00   49.00  20.00    28.00    19.00   
t_clust_size   786.00   330.00  13.00   49.00  20.00    28.00    19.00   
p_overlap_per    6.78    77.11   0.45   13.10   2.56    77.11    77.11   
r_overlap_per   11.86    20.34   3.39   22.03   3.39     3.39     5.08   
s_overlap_per   54.29    35.71   1.43    2.86   1.43     2.86     0.00   
t_overlap_per   37.50    47.50   2.50    5.00   0.00     5.00     0.00   

                     7       8  
p_cl

In [82]:
# dict_result2 = {}
# for temp_key in dict_cluster_asso_chain.keys():
#     temp_res_dict = {}
#     #
#     p_clust_id = dict_cluster_asso_chain[temp_key]["p"]
#     r_clust_id = dict_cluster_asso_chain[temp_key]["r"]
#     s_clust_id = dict_cluster_asso_chain[temp_key]["d"]
#     t_clust_id = dict_cluster_asso_chain[temp_key]["d"]
#     #
#     p_clust_size = dict_entity_cluster_info["p"]['num'][p_clust_id]
#     r_clust_size = dict_entity_cluster_info["r"]['num'][r_clust_id]
#     s_clust_size = dict_entity_cluster_info["d"]['num'][s_clust_id]
#     t_clust_size = dict_entity_cluster_info["d"]['num'][t_clust_id]
#     #
#     p_overlap_per = get_overlap_percent2(dict_entity_cluster_info,"p",p_clust_id,data_dict2)
#     r_overlap_per = get_overlap_percent2(dict_entity_cluster_info,"r",r_clust_id,data_dict2)
#     s_overlap_per = get_overlap_percent2(dict_entity_cluster_info,"s",s_clust_id,data_dict2)
#     t_overlap_per = get_overlap_percent2(dict_entity_cluster_info,"t",t_clust_id,data_dict2)
#     #
#     temp_res_dict["p_clust_id"] = p_clust_id
#     temp_res_dict["r_clust_id"] = r_clust_id
#     temp_res_dict["s_clust_id"] = s_clust_id
#     temp_res_dict["t_clust_id"] = t_clust_id
#     #
#     temp_res_dict["p_clust_size"] = p_clust_size
#     temp_res_dict["r_clust_size"] = r_clust_size
#     temp_res_dict["s_clust_size"] = s_clust_size
#     temp_res_dict["t_clust_size"] = t_clust_size
#     #
#     temp_res_dict["p_overlap_per"] = p_overlap_per
#     temp_res_dict["r_overlap_per"] = r_overlap_per
#     temp_res_dict["s_overlap_per"] = s_overlap_per
#     temp_res_dict["t_overlap_per"] = t_overlap_per
#     #
#     dict_result2[temp_key] = temp_res_dict

In [83]:
# df_result2 = pd.DataFrame(dict_result2)
# pp.pprint(df_result2)
# print("#")
# df_result2 = pd.DataFrame(dict_result2).T
# #pp.pprint(df_result[["p_overlap_per","r_overlap_per","s_overlap_per","t_overlap_per"]].T)

In [84]:
max_cc,min_cc

(2, 1)

In [85]:
pp.pprint(df_result1[["p_overlap_per","r_overlap_per","s_overlap_per","t_overlap_per"]].T)

                   0      1     2      3     4      5      6      7     8
p_overlap_per   6.78  77.11  0.45  13.10  2.56  77.11  77.11  77.11  6.78
r_overlap_per  11.86  20.34  3.39  22.03  3.39   3.39   5.08   3.39  3.39
s_overlap_per  54.29  35.71  1.43   2.86  1.43   2.86   0.00   1.43  0.00
t_overlap_per  37.50  47.50  2.50   5.00  0.00   5.00   0.00   2.50  0.00


In [86]:
#pp.pprint(df_result2[["p_overlap_per","r_overlap_per","s_overlap_per","t_overlap_per"]].T) #computing this, does not make sense

In [87]:
#

In [88]:
df_result1.T

,0,1,2,3,4,5,6,7,8
p_clust_id,1.00,0.00,2.00,3.00,4.00,0.00,0.00,0.00,1.00
r_clust_id,6.00,5.00,8.00,9.00,8.00,8.00,0.00,3.00,8.00
s_clust_id,0.00,1.00,2.00,3.00,4.00,5.00,6.00,7.00,8.00
t_clust_id,0.00,1.00,2.00,3.00,4.00,5.00,6.00,7.00,8.00
p_clust_size,548.00,4948.00,15.00,329.00,50.00,4948.00,4948.00,4948.00,548.00
r_clust_size,38.00,101.00,22.00,155.00,22.00,22.00,54.00,12.00,22.00
s_clust_size,786.00,330.00,13.00,49.00,20.00,28.00,19.00,57.00,19.00
t_clust_size,786.00,330.00,13.00,49.00,20.00,28.00,19.00,57.00,19.00
p_overlap_per,6.78,77.11,0.45,13.10,2.56,77.11,77.11,77.11,6.78
r_overlap_per,11.86,20.34,3.39,22.03,3.39,3.39,5.08,3.39,3.39


In [89]:
data_dict["gt_case1_part"]["dict_gt_entity_size"]

{'p': 664, 'r': 59, 's': 70, 't': 40}